In [ ]:
!git clone https://github.com/jwkirchenbauer/lm-watermarking.git
!pip install -U evaluate

Cloning into 'lm-watermarking'...
remote: Enumerating objects: 328, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 328 (delta 17), reused 5 (delta 5), pack-reused 289
Receiving objects: 100% (328/328), 11.99 MiB | 15.06 MiB/s, done.
Resolving deltas: 100% (96/96), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.8 MB/s eta 0:00:00


In [ ]:
# import statements
import sys
sys.path.append("lm-watermarking/")
from datasets import load_dataset
import evaluate
from functools import partial
from transformers import AutoModelForCausalLM, AutoTokenizer, LogitsProcessorList
from extended_watermark_processor import WatermarkDetector, WatermarkLogitsProcessor
import torch
import pandas as pd
from typing import Callable, Dict
from transformers import LogitsProcessorList, PreTrainedTokenizerBase as Tokenizer
import time

In [ ]:
# Load Xsum dataset from huggingface
dataset_name = 'xsum'
dataset = load_dataset(dataset_name, split='train', streaming=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [ ]:
# function to add index to each row of the dataset and
# add a sentence that indicates a summary generation to the document
def add_idx(row, idx):
    row.update({"idx":idx, "document": row['document']+"\nDon’t Give Me the Details, Just the Summary!"})
    return row

# function to tokenize the document for each row of the dataset
def tokenize_data(row, idx, tokenizer):
    inputs = tokenizer.encode(row["document"], return_tensors="pt", truncation=True)
    row.update({"inputs": inputs})
    return row

# function to complete the generations for each row based
def generate_completions(row, idx, max_new_tokens, tokenizer, model, no_bl_partial, w_bl_partial, bl_processor_list):

    input_tokens = row["inputs"] # input tokens
    original_input = row["document"] # actual document string

    # Disable gradient calculations for generation
    with torch.no_grad():
        attempts = 0
        max_attempts = 10
        operation_successful = False
        while not operation_successful and attempts < max_attempts:
            attempts += 1

            # generation without watermarking
            gen_start = time.time()
            bl_free_output = no_bl_partial(input_tokens)
            row["time_gen_no_bl"] = time.time() - gen_start

            # generation with watermarking
            gen_start = time.time()
            bl_included_output = w_bl_partial(input_tokens)
            row["time_gen_w_bl"] = time.time() - gen_start

            if bl_processor_list:
                # storing spike entropies for each generation
                row["entropy_values"] = bl_processor_list[0]._get_and_clear_stored_spike_ents() if bl_processor_list[0].spike_entropies else None

            try:
                # decode the generated tokens into text
                decoded_output_no_bl = tokenizer.batch_decode(bl_free_output, skip_special_tokens=True)[0]
                row["decoded_no_bl"] = decoded_output_no_bl.replace(original_input, "")

                decoded_output_w_bl = tokenizer.batch_decode(bl_included_output, skip_special_tokens=True)[0]
                row["decoded_w_bl"] = decoded_output_w_bl.replace(original_input, "")

                operation_successful = True
            except Exception as e:
                if attempts == max_attempts:
                    # Decoding failed after several attempts, defaulting outputs to empty.
                    row["decoded_no_bl"] = ""
                    row["decoded_w_bl"] = ""
                    if bl_processor_list:
                        row["entropy_values"] = [] if bl_processor_list[0].spike_entropies else None

    row["tokens_generated_no_bl"] = bl_free_output.shape[1] - input_tokens.shape[1]
    row["tokens_generated_w_bl"] = bl_included_output.shape[1] - input_tokens.shape[1]
    row["speed_no_bl"] = row["time_gen_no_bl"] / row["tokens_generated_no_bl"]
    row["speed_w_bl"] = row["time_gen_w_bl"] / row["tokens_generated_w_bl"]

    return row

# function to get the watermark logits processor
def logits_processor(tokenizer, gamma, delta, seeding_scheme, vocab_list):
    logits = WatermarkLogitsProcessor(vocab=vocab_list,
                                                gamma=gamma,
                                                delta=delta,
                                                seeding_scheme="selfhash", store_spike_ents=True)
    return LogitsProcessorList([logits])

In [ ]:
# function to take the generated sequence and runs it through a detection algorithm
def detect_watermark(row, idx, tokenizer, model, vocab_list, gamma=0.25, z_thres=4.0):
    watermark_detector = WatermarkDetector(vocab=vocab_list,
                                            gamma=gamma, # should match original setting
                                            seeding_scheme="selfhash", # should match original setting
                                            device=model.device, # must match the original rng device type
                                            tokenizer=tokenizer,
                                            z_threshold=z_thres,
                                            normalizers=[],
                                            ignore_repeated_ngrams=True)
    results = watermark_detector.detect(row['decoded_w_bl'])
    row['z_score'] = results['z_score']
    row['prediction'] = results['prediction']
    return row

# function to calculate perplexity for each generation of the watermarked text
def cal_perplexity(row, idx, model_name):
    perplexity = evaluate.load("perplexity", module_type="metric")
    results = perplexity.compute(model_id=model_name,
                                add_start_token=False,
                                predictions=[row['decoded_w_bl']])
    row['perplexity'] = results['perplexities'][0]
    return row

In [ ]:
# function to get the model and tokenizer
def get_model_tokenizer(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    return model, tokenizer

In [ ]:
# used models
model_name = ["facebook/opt-1.3b", "facebook/opt-2.7b"]

In [ ]:
# fetching genration model and tokenizer
model, tokenizer = get_model_tokenizer(model_name[0])

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
# number of indices to process for each pair of delta and gamma
limit_indices = 25

The pipeline is run twice, once with multinomial sampling and once with beam search. With delta=10.0 and gamma=0.25 for each generation. The two files generated through this pipeline are stored in the generated_files folder in the drive link provided in the report.

In [ ]:
delta = 10.0
gamma = 0.25
n_beams = 8
# pipeline for experiment file generation
indexed_dataset = dataset.map(add_idx, batched=False, with_indices=True)
shuffled_dataset = indexed_dataset.shuffle(seed=1234, buffer_size=10000)

gen_tok = partial(tokenize_data, tokenizer=tokenizer)
tokenized_dataset = shuffled_dataset.map(gen_tok,
                                            batched=False,
                                            with_indices=True)


# perform generation by calling the models
no_bl = partial(model.generate, max_new_tokens=200, do_sample=True, top_k=0, temperature=0.7 , num_beams=n_beams)
vocab_list = list(tokenizer.get_vocab().values())
logits_list = logits_processor(tokenizer=tokenizer, gamma=gamma, delta=delta, seeding_scheme="selfhash", vocab_list=vocab_list)
w_bl = partial(model.generate, logits_processor=logits_list, max_new_tokens=200, do_sample=True, top_k=0, temperature=0.7 , num_beams=n_beams)

# map the generation function after shuffling the dataset
gen_com = partial(generate_completions, max_new_tokens=200, tokenizer=tokenizer, model=model, no_bl_partial=no_bl, w_bl_partial=w_bl, bl_processor_list=logits_list)
generations_dataset = tokenized_dataset.map(gen_com,
                                                batched=False,
                                                with_indices=True)

# map the detection function after generating the watermarked and non-watermarked sequences
watermark_detect = partial(detect_watermark, tokenizer=tokenizer, model=model, vocab_list=vocab_list, gamma=gamma, z_thres=4.0)
watermark_detections = generations_dataset.map(watermark_detect, batched=False, with_indices=True)

# map the perplexity function after running detection algorithm
ppl_cal = partial(cal_perplexity, model_name=model_name[1])
ppl_dataset = watermark_detections.map(ppl_cal, batched=False, with_indices=True)

# executes the pipeline for set number of indices for each row and then stores the results in a csv as specified below
ds_iterator = iter(ppl_dataset)
for i in range(limit_indices):
    df = pd.DataFrame([next(ds_iterator)])
    if i==0:
        df.to_csv(f"drive/MyDrive/delta_{delta}_gamma_{gamma}_beams{n_beams}_xsum.csv", index=False)
    else:
        df.to_csv(f"drive/MyDrive/delta_{delta}_gamma_{gamma}_beams{n_beams}_xsum.csv", mode='a', index=False, header=False)
    print(i+1)
print(f"delta_{delta}_gamma_{gamma}_beams{n_beams} Done ........................................")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.30G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

...

23


  0%|          | 0/1 [00:00<?, ?it/s]

24


  0%|          | 0/1 [00:00<?, ?it/s]

25
delta_10.0_gamma_0.25_beams8 Done ........................................
